## Representing Traits on Trees

Now that we have a handle on phylogenetic trees and an idea of how to represent them, we can add in traits to those trees.

## Species can share traits due to homology or convergence

When species share a trait due to inheritance from a common ancestor, that is called **homology** and the shared trait is called a **homologous trait**. When species instead share a trait because it evolved more than once on the tree, that is called **convergent evolution** and the shared trait is known as a **homoplasy**. 

An example of homologous traits vs. convergently evolved traits is shown below:

<figure caption-side="bottom">
<img src="./resources/homology_vs_convergence.png" width="500" style="margin:5px 25px" description="A tree showing a homologous trait arising in an ancestor and being inherited by several of its descendants, and another tree showing the same trait evolving separately in two totally different parts of the tree." > 
<figcaption align="left"><b>Fig. 1</b> - Homologous vs. convergently evolved traits. <br></figcaption>
</figure>


Whether a particular traits is a homologous trait or a homoplasy depends on the tree. Indeed, modern methods for inferring trees test millions of possible trees to find the ones that do the best job of explaining as many shared traits as possible by homology.

## Applications of trees with traits

Trees that have trait information associated with them have many applications.

## Ancestral State Reconstruction

The goal of ancestral state reconstruction (ASR) is to determine the traits of ancestral organisms by comparative analysis of modern organisms. More formally, given annotations for the traits of tip nodes, ancestral state reconstruction attempts to reconstruct the traits at internal nodes.


#### The Fitch Parsimony algorithm for Ancestral State Reconstruction.

The Fitch Parsimony algorithm is designed to find a most parsimonious ancestral state reconstruction for ancestral character states.




##### Review of set unions and intersections

[TODO]

##### Implementing Fitch Parsimony in Python

[TODO]

##### Preorder and postorder traversals

To execute the Fitch Parsimony algorithm we'll have to have a way to visit all the nodes on the tree. Unlike something linear like a list, there are [several different reasonable ways](https://www.geeksforgeeks.org/tree-traversals-inorder-preorder-and-postorder/) to visit all the nodes in the tree:
* one might start at the root and work upwards, visiting more ancestral nodes first and only visiting descendant nodes when all their ancestors hvae been visited (preorder traversal)
* one might start at the tips, and only visit ancestral nodes after all their descendants have been visited (postorder traversal).




#### Trait prediction

[TODO]

## Molecular Clock Analysis

Molecular clock analysis uses the rate at which traits change over time as a sort of clock. For example, we might expect that if we compare the same snippet of DNA across many species, a pair of species that are 99% similar in DNA sequence in that gene probably diverged much more recently than a pair that are only 76% similar. However, such methods must be careful to account for several factors. 

First, some changes in DNA sequence will be hidden. For example, imagine a given nucleotide mutated from A to T and then, some years later, back from T to A. This was two changes in the DNA sequence, but a later comparative analysis would see 0 changes! For similar reasons, even two random sequences with equal base frequencies will appear to share 25% sequence identity. 

A second point where caution is warrented is that the rate of evolution may change in different parts of the tree. For example, if a mutation renders the proof-reading machinery that tries to correct mutations more or less strict, it can decrease or increase the mutation rate in that part of the tree. **Relaxed molecular clocks** try to account for this by allowing different rates of evolution to occur in different parts of the tree.

## Implementing a simple molecular clock in python

Given a set of divergence times, a tree, and a set of sequences we will write code that tries to calibrate a simple molecular clock that can then be used to estimate the divergence times for new species pairs.

[TODO]

## Exercises

**Exercise 1.** Build the tree illustrating homology and convergence shown at the top of this section using PhyloNode objects with traits. Test that the traits are on the right nodes.

**Exercise 2.** Write an isHomologous method for the PhyloNode class that tests if a given trait is homologous between two species. Test it on the tree from Exercise 1 and see if you get the expected result. *Hint*: if a trait is homologous between two species, then it implies the trait evolved before the species diverged into separate lineages. Therefore,  all the shared ancestors of those two species should have the trait.

**Exercise 3.** Why must we use postorder rather than preorder traversal when performing the Fitch Parsimony downpass?


## Further Reading

[1] ["Tree Traversals (inorder,preorder and postorder)"](https://www.geeksforgeeks.org/tree-traversals-inorder-preorder-and-postorder/). Geeks for Geeks (Contributed by: danielbritten & andrew1234), 